Training a model in AML
1. step up an azure machine learning workspace
2. prepare your data: gather and preprocess your data
3. define your compute environment: options include AML compute, azure databricks and azure virtual machines
4. choose your model type: based on your problem doman and data. it can be a classification, regression, clustering model or any type
5. write your training script: it include loading the data, defining the model architecture, training the model and evaluating its perfomance
6. create an experiment: they are used to track the training process and results. create an experiment to log metrics, outputs and other relevant information during model training
7. Run your script training script: submit your training script to the choosen  compute environment to start training process.
8. monitor training progress: you can track metrics, view logs and troubleshoot any issue that may arise.
9. evaluate your model: evaluate your model using validation of data or other techniques. model testing makes it easy to deploy and manage your model in production environments.
10. Deploy your trained model as a web service or container to make predictions on new data.
11. Monitor and maintain your deployed model 

1. set up AZML workspace.
2.prepare data and write a script

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# load the data 
iris_df = pd.read_csv("https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv")
# split the data into features and target
X = iris_df.drop('species', axis=1)
y = iris_df['species']
# split ,the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#train a random forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
#evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('accuracy', accuracy)

accuracy 1.0


step 3. Define the compute environment

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import AmlCompute, ComputeTarget

# Load workspace
workspace = Workspace.from_config()

# Define Azure Machine Learning Compute
compute_name = "cpu-cluster"
compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS2_V2", max_nodes=4)
compute_target = ComputeTarget.create(workspace, compute_name, compute_config)
compute_target.wait_for_completion(show_output=True)


step 4: create an experiment and run training script

In [6]:
from azureml.core import Experiment, ScriptRunConfig
#create an experiment
experiment_name = "iris_classification"
experiment = Experiment(workspace, experiment_name)

#create a script run configuration
src = ScriptRunConfig(source_directory=".", script="train.py", compute_target=compute_target)

#submit the experiment
run = experiment.submit(src)
run.wait_for_completion(show_output=True)

AttributeError: module 'azureml.core.workspace' has no attribute 'service_context'

step 5: register the model

In [5]:
from azureml.core import Model
#register the model
model_name = "iris-classifier"
model_path = "outputs/model.pkl"
model = run.register_model(model_name=model_name, model_path=model_path)

NameError: name 'run' is not defined

step 6: deploy the model

In [ ]:
from azureml.core import Environment, Model
from azureml.core.model import InferenceConfig
from azureml.core.model import InferenceConfig

#define environment
env = Environment.from_conda_specification(name="sklearn-env", file_path="Environment.yml")
#define inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=env)

#deploy model as a web service

service_name = "iris-classifier-service"
deployment_config = AciWebservice.deploy_configuration